In [ ]:
import os
print(os.getcwd())


In [ ]:
os.chdir('C:/Users/chira/Downloads/athira-downloads/smartviz_chatbot_project')


In [ ]:
!pip install seaborn 
!pip install matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine


# Load from local CSVs (placed in the 'data/' folder)
hierarchy_df = pd.read_csv('data/metrics_app_hierarchy_202506111454_backup.csv')
timeagg_df = pd.read_csv('data/metrics_app_timeaggregated_202507091430_backup-1752067848951_cleaned.csv')




In [ ]:
hierarchy_df.shape

In [ ]:
timeagg_df.shape

In [ ]:
11+14

In [ ]:
# Display basic info and first few rows
print("\n Hierarchy Dataset:")
print(hierarchy_df.info())
print(hierarchy_df.head())

In [ ]:
print("\n Time-Aggregated Dataset:")
print(timeagg_df.info())
print(timeagg_df.head())

In [ ]:
# 1. Missing values
print(" Missing Values in Hierarchy:")
print(hierarchy_df.isnull().sum())

print("\n Missing Values in Time-Aggregated:")
print(timeagg_df.isnull().sum())

# 2. Convert to datetime & check errors
timeagg_df['start_time'] = pd.to_datetime(timeagg_df['start_time'], errors='coerce')
print("\n Converted start_time to datetime")
print("Missing or invalid start_time:", timeagg_df['start_time'].isnull().sum())

# 3. Unique metrics and overall date range
print("\n Unique Metrics:", timeagg_df['metric_name'].nunique())
print("Metrics:", timeagg_df['metric_name'].unique())
print("Date Range:", timeagg_df['start_time'].min(), "→", timeagg_df['start_time'].max())

# 4. Basic descriptive stats (value column)
print("\n Summary Statistics of 'value':")
print(timeagg_df['value'].describe())


In [ ]:
def detect_outliers_iqr(df, metric_column='value', group_by='metric_name'):
    outlier_summary = []

    for metric in df[group_by].unique():
        sub = df[df[group_by] == metric]
        q1 = sub[metric_column].quantile(0.25)
        q3 = sub[metric_column].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        outliers = sub[(sub[metric_column] < lower) | (sub[metric_column] > upper)]
        pct_outliers = 100 * len(outliers) / len(sub)

        outlier_summary.append({
            "metric_name": metric,
            "total_values": len(sub),
            "outliers": len(outliers),
            "%_outliers": round(pct_outliers, 2),
            "value_min": sub['value'].min(),
            "value_max": sub['value'].max(),
            "iqr_lower": round(lower, 2),
            "iqr_upper": round(upper, 2),
        })

    return pd.DataFrame(outlier_summary)

# Run the detection
outlier_report = detect_outliers_iqr(timeagg_df)
outlier_report


In [ ]:
#  Merge the two datasets on geometry_id
merged_df = timeagg_df.merge(hierarchy_df, on='geometry_id', how='left', validate='many_to_one')

#  Check for success and nulls after merge
print("\n Merged Dataset Info:")
print(merged_df.info())
print("\n Missing values after merge:")
print(merged_df.isnull().sum())

#  preview
print("\n Preview of merged data:")
print(merged_df.head())


In [ ]:
print(set(timeagg_df.columns).intersection(set(hierarchy_df.columns)))


In [ ]:
# Drop rows without display_name (important for graph + insight generation)
merged_df = merged_df.dropna(subset=['display_name'])

# Optional: also drop rows missing start_time or value
merged_df = merged_df.dropna(subset=['start_time', 'value'])

# Fill in other less critical nulls with safe defaults
merged_df['name'] = merged_df['name'].fillna('Unknown')
merged_df['parent'] = merged_df['parent'].fillna('Unknown')
merged_df['hierarchy_type'] = merged_df['hierarchy_type'].fillna('Unknown')
merged_df['longitude'] = merged_df['longitude'].fillna(0)
merged_df['latitude'] = merged_df['latitude'].fillna(0)


In [ ]:
merged_df['start_time'] = pd.to_datetime(merged_df['start_time'], errors='coerce', utc=True)
merged_df = merged_df.dropna(subset=['start_time'])

# Extract datetime features
merged_df['hour'] = merged_df['start_time'].dt.hour
merged_df['dayofweek'] = merged_df['start_time'].dt.dayofweek
merged_df['is_weekend'] = merged_df['dayofweek'] >= 5
merged_df['month'] = merged_df['start_time'].dt.month


In [ ]:
def remove_outliers_iqr(df, value_col='value', group_col='metric_name'):
    cleaned_df = pd.DataFrame()
    removed_summary = []

    for metric, group in df.groupby(group_col):
        if metric == 'Occupancy':
            # Skip filtering for Occupancy
            inliers = group
            outliers = pd.DataFrame()
        else:
            q1 = group[value_col].quantile(0.25)
            q3 = group[value_col].quantile(0.75)
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr

            inliers = group[(group[value_col] >= lower) & (group[value_col] <= upper)]
            outliers = group[(group[value_col] < lower) | (group[value_col] > upper)]

        removed_summary.append({
            'metric_name': metric,
            'original_count': len(group),
            'removed_outliers': len(outliers),
            'percent_removed': round(100 * len(outliers) / len(group), 2)
        })

        cleaned_df = pd.concat([cleaned_df, inliers], ignore_index=True)

    removed_df = pd.DataFrame(removed_summary)
    return cleaned_df, removed_df


In [ ]:
merged_df_cleaned, outlier_stats = remove_outliers_iqr(merged_df)

# View summary of removed outliers per metric
print(outlier_stats.sort_values(by='percent_removed', ascending=False))

# Optional: inspect shape before and after
print(f"Original shape: {merged_df.shape}")
print(f"Cleaned shape: {merged_df_cleaned.shape}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
sns.set(style="whitegrid")

# Metrics to visualize
metrics_to_plot = ['Occupancy', 'co2', 'temp', 'peopleMotion']

# Plot boxplots for each selected metric
for metric in metrics_to_plot:
    plt.figure(figsize=(8, 5))
    sns.boxplot(
        x='metric_name',
        y='value',
        data=merged_df_cleaned[merged_df_cleaned['metric_name'] == metric],
        color='lightblue'
    )
    plt.title(f"Boxplot of {metric} (After Outlier Removal)")
    plt.xlabel("")
    plt.ylabel("Value")
    plt.tight_layout()
    plt.show()


In [ ]:
hourly_occupancy = merged_df_cleaned.groupby(merged_df_cleaned['start_time'].dt.hour)['value'].mean()
hourly_occupancy.plot(kind='bar', figsize=(10,5), title='Average Occupancy by Hour')


In [ ]:
merged_df_cleaned['is_peak_hour'] = merged_df_cleaned['start_time'].dt.hour.between(7, 10)


In [ ]:
def extract_room_type(name):
    name = str(name).lower()
    if 'seminar' in name:
        return 'Seminar Room'
    elif 'lecture' in name:
        return 'Lecture Hall'
    elif 'restroom' in name or 'toilet' in name:
        return 'Restroom'
    elif 'corridor' in name:
        return 'Corridor'
    elif 'lobby' in name:
        return 'Lobby'
    elif 'lab' in name:
        return 'Lab'
    elif 'meeting' in name:
        return 'Meeting Room'
    else:
        return 'Other'

merged_df_cleaned['room_type'] = merged_df_cleaned['display_name'].apply(extract_room_type)


In [ ]:
# Compute average occupancy per room
room_avg = merged_df_cleaned.groupby('display_name')['value'].mean()

# Find median occupancy across all rooms
median_occupancy = room_avg.median()

# Define high or low usage per room
room_usage_category = room_avg.apply(lambda x: 'High' if x > median_occupancy else 'Low')

# Map it back to the main DataFrame
merged_df_cleaned['usage_category'] = merged_df_cleaned['display_name'].map(room_usage_category)


In [ ]:
print(merged_df_cleaned[['start_time', 'is_peak_hour', 'room_type', 'usage_category']].head())


In [ ]:

# Optional: Visualize occupancy by room type
plt.figure(figsize=(10, 6))
sns.barplot(data=merged_df_cleaned[merged_df_cleaned['metric_name'] == 'Occupancy'],
            x="room_type", y="value", estimator="mean", ci=None, palette="viridis")
plt.title("Average Occupancy by Room Type")
plt.ylabel("Average Occupancy")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Filter only Occupancy metric
occupancy_df = merged_df_cleaned[merged_df_cleaned['metric_name'] == 'Occupancy']

# Compute average occupancy per room
room_avg_occupancy = occupancy_df.groupby('display_name')['value'].mean().sort_values(ascending=False)

# Plot average occupancy for each room
plt.figure(figsize=(14, 6))
sns.barplot(x=room_avg_occupancy.index, y=room_avg_occupancy.values, palette="coolwarm")
plt.xticks(rotation=90)
plt.title("Average Occupancy per Room")
plt.ylabel("Average Occupancy")
plt.xlabel("Room Name")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Compute average occupancy per room
occupancy_by_room = (
    merged_df_cleaned[merged_df_cleaned['metric_name'] == 'Occupancy']
    .groupby('display_name')['value']
    .mean()
    .sort_values(ascending=False)
)

# Step 2: Keep top 10, group the rest into "Other (Low Usage)"
top_n = 10
top_rooms = occupancy_by_room.head(top_n)
other_sum = occupancy_by_room.iloc[top_n:].mean()  # use mean or sum depending on message
top_rooms['Other (Low Usage)'] = other_sum

# Step 3: Convert to DataFrame for plotting
plot_df = top_rooms.reset_index()
plot_df.columns = ['Room Name', 'Average Occupancy']

# Step 4: Add usage category for coloring
plot_df['Usage Category'] = ['High Usage'] * top_n + ['Low Usage']

# Step 5: Plot
plt.figure(figsize=(12, 6))
sns.barplot(data=plot_df, x='Room Name', y='Average Occupancy', hue='Usage Category', dodge=False, palette={'High Usage': 'green', 'Low Usage': 'steelblue'})
plt.title("Top 10 Rooms by Average Occupancy (Others grouped)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
merged_df_cleaned[merged_df_cleaned['metric_name'] == 'Occupancy'].describe()


In [ ]:
#after box plot aarunu
# 1. Save locally
merged_df_cleaned.to_csv('data/metrics_app_merged_cleaned.csv', index=False)
print(" Cleaned dataset saved to 'data/metrics_app_merged_cleaned.csv'")

# 2. Save to PostgreSQL
from sqlalchemy import create_engine

# Database connection details
DB_USER = 'postgres'
DB_PASS = 'SmartViz2025'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'smartviz'

# Create the connection engine
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

# Push to Postgres, replacing existing table
merged_df_cleaned.to_sql('metrics_app_merged', engine, if_exists='replace', index=False)
print(" Cleaned dataset saved to PostgreSQL table 'metrics_app_merged'")


In [ ]:
merged_df_cleaned.shape